In [1]:
import os
import sys

sys.path.append("/home/dc755/Conjecturing/")
sys.path.append("/home/dc755/Conjecturing/conjecture_generation/")
sys.path.append("/home/dc755/Conjecturing/conjecture_generation/src/")
WORK_DIR = os.getcwd()
print(sys.path)

from utils.create import load_template
from utils.json_utils import write_to_new_json, load_json_as_list
from conjecture_generation.src.domain import Domain
from conjecture_generation.src.func_parameterization import FunctionParameterization
from conjecture_generation.src.functions import exponential_box
from conjecture_generation.src.conj_gen import find_new_conj, sgd_signum_loss, save_conj
from initial_functions import poly_box
import dsp_functions

['/home/dc755/Conjecturing', '/home/dc755/.conda/envs/conj_env/lib/python310.zip', '/home/dc755/.conda/envs/conj_env/lib/python3.10', '/home/dc755/.conda/envs/conj_env/lib/python3.10/lib-dynload', '', '/home/dc755/.conda/envs/conj_env/lib/python3.10/site-packages', '/home/dc755/Conjecturing/', '/home/dc755/Conjecturing/conjecture_generation/', '/home/dc755/Conjecturing/conjecture_generation/src/']
[exp, Abs]


In [2]:
domain = Domain(
    func_box=poly_box,
    max_degree=1,
    latent_dim=1,
    evaluation_dimension=10,
    testset_size=100,
)

In [3]:

[x_var, combs, eval_point] = domain.generate_evaluation_domain_continuous(
    pnt_start=-3, pnt_end=3, n_pnts=100
)

[y_symbs, num_pars, func_evals, eval_data] = domain.generate_hypothesis_space(
    vars=x_var, cmbs=combs, eval_points=eval_point
)

func_param = FunctionParameterization(num_pars=num_pars, y_symbs=y_symbs)
print(f"Initial θ vector is: {func_param.θ}")

no_of_tries = 10000
counter = no_of_tries
while counter > 0:
    [initial_conj, initial_loss, flag, sig_val] = find_new_conj(
        num_pars=num_pars,
        eval_data=eval_data,
        y_symbs=y_symbs,
        func_evals=func_evals,
    )
    print(
        f"initial conj number: {no_of_tries - counter} is: {initial_conj}..associated loss is: {initial_loss}"
    )
    initial_conj = str(initial_conj)
    if not flag:
        print("Conjecture is initial conjecture: ", initial_conj)
        counter -= 1
    else:
        break

if flag:
    print("Actual learning needed")
    [final_conj, final_loss, not_contains_zero] = sgd_signum_loss(
        y_symbs=y_symbs,
        training_data=eval_data,
        func_param=func_param,
        sig_val=sig_val,
        batch_prct=0.1,
        num_epochs=100,
        θ_differential=100,
        learn_rate=10,
        initial_loss=initial_loss,
        loss_type="signum",
        comp_number=1,
        num_pars=num_pars,
    )

    sub_list = zip(y_symbs, func_evals)
    final_conj = final_conj.subs(sub_list)
    final_conj = str(final_conj)
    greater_than, less_than = [">", "<"]
    if not not_contains_zero:
        if sig_val > 0:
            print(f"The final conjecture is: {final_conj} >= 0")
        else:
            final_conj = final_conj.replace(">", "<")
            print(f"The final conjecture is: {final_conj} <= 0")

    if sig_val > 0:
        print(f"The final conjecture is: {final_conj} > 0")
    else:
        final_conj = final_conj.replace(">", "<")
        print(f"The final conjecture is: {final_conj} < 0")

    # file_saved.append(local_save_conj(initial_conj=initial_conj, final_conj=final_conj, sig_val=sig_val))
    print("Final conjecture: ", final_conj)

    print(f"Found {no_of_tries - counter + 1} conjectures in total!")

else:
    print("Couldn't find conjecture needing training but saved 100 conjectures!")

100%|██████████| 100/100 [00:00<00:00, 4400.47it/s]


Initial θ vector is: [8.0, -1.0, 4.0]
initial conj number: 0 is: 4.00000000000000..associated loss is: 0.0
Conjecture is initial conjecture:  4.00000000000000
initial conj number: 1 is: 16.0*x_1**2 + 2.0..associated loss is: 0.0
Conjecture is initial conjecture:  16.0*x_1**2 + 2.0
initial conj number: 2 is: 6.0*x_1**2 + 14.0..associated loss is: 0.0
Conjecture is initial conjecture:  6.0*x_1**2 + 14.0
-0.72
initial conj number: 3 is: 11.0*x_1**2 - 2.0..associated loss is: 0.23193856000000002
Actual learning needed
Epoch number: 0...initial_θ_vector is: [ 8. -1.  4.]..initial_loss is: 0.23193856000000002
Differential loss is: [0. 0. 0.]
Epoch number: 1..θ_vector is: [ 1. -1.  4.]..loss is: 0.0
conj_func < 0:  -2.0*y_1 - 4.0
The final conjecture is: -2.0*x_1**2 - 4.0 < 0
Final conjecture:  -2.0*x_1**2 - 4.0
Found 4 conjectures in total!
